In [1]:
import pandas as pd
import psycopg2
import tmdbsimple as tmdb
tmdb.API_KEY = '18814be112b8b0167bc919c307bd596e'


In [2]:
df = pd.read_csv("movieid.csv", header=None)

In [3]:
#df = df.drop(columns="Unnamed: 0")

In [4]:
id_list = df[1].head(30)

In [91]:
id_list

0      tt0288772
1      tt9042520
2      tt2911668
3      tt3489334
4      tt0814314
5     tt10778974
6      tt8365032
7      tt0082287
8      tt1801844
9      tt0268932
10     tt0413323
11     tt0240674
12     tt6278008
13     tt0399845
14     tt1701226
15     tt1754314
16     tt7642434
17     tt6839248
18     tt0778597
19     tt0090822
20     tt0839856
21     tt7785128
22     tt2189778
23     tt4556796
24     tt1194445
25     tt0111066
26     tt7310722
27     tt3845730
28     tt1693760
29     tt0288035
Name: 1, dtype: object

In [39]:
df = df.head(30)
df['m_id']

0      tt0288772
1      tt9042520
2      tt2911668
3      tt3489334
4      tt0814314
5     tt10778974
6      tt8365032
7      tt0082287
8      tt1801844
9      tt0268932
10     tt0413323
11     tt0240674
12     tt6278008
13     tt0399845
14     tt1701226
15     tt1754314
16     tt7642434
17     tt6839248
18     tt0778597
19     tt0090822
20     tt0839856
21     tt7785128
22     tt2189778
23     tt4556796
24     tt1194445
25     tt0111066
26     tt7310722
27     tt3845730
28     tt1693760
29     tt0288035
Name: m_id, dtype: object

In [51]:
def remove_tt(col):
    id_no_tt = []
    for id in col:        
        rep = id.replace("tt", "")
        id_no_tt.append(rep)
    return id_no_tt

In [74]:
id_list_other = pd.DataFrame({'ids': remove_tt(df['m_id'])})
id_list_other

,ids
0,0288772
1,9042520
2,2911668
3,3489334
4,0814314
5,10778974
6,8365032
7,0082287
8,1801844
9,0268932


In [75]:
def make_list(id_list_other):
    id_list_int = ""
    for i in id_list_other.itertuples(index=False):
        id_list_int += str((i.ids)) + ", "
    # remove hanging comma
    id_list_int = id_list_int[:-2]
    return id_list_int

In [76]:
id_list_int = make_list(id_list_other)
id_list_int

'0288772, 9042520, 2911668, 3489334, 0814314, 10778974, 8365032, 0082287, 1801844, 0268932, 0413323, 0240674, 6278008, 0399845, 1701226, 1754314, 7642434, 6839248, 0778597, 0090822, 0839856, 7785128, 2189778, 4556796, 1194445, 0111066, 7310722, 3845730, 1693760, 0288035'

In [105]:
def test_query():
    try:
        connection = psycopg2.connect(user="lambda",
                                      password="lambdaschoolgroa",
                                      host="localhost",
                                      port="54320",
                                      database="groa")

        #print("Using Python variable in PostgreSQL select Query")
        cursor = connection.cursor()
        postgreSQL_select_Query = """SELECT movie_id, poster_url FROM imdb_movies ORDER BY movie_id DESC LIMIT 30;"""
        cursor.execute(postgreSQL_select_Query)
        mobile_records = cursor.fetchall()
        mobile_records_df = pd.read_sql_query(postgreSQL_select_Query, connection)
        return mobile_records_df
    finally:
        # closing database connection
        if (connection):
            cursor.close()
            connection.close()
            #print("PostgreSQL connection is closed \n")
            
test_query()

,movie_id,poster_url
0,9916730,None
1,9916680,None
2,9916538,None
3,9916190,None
4,9916186,None
5,9916160,None
6,9916134,None
7,9916132,None
8,9916124,None
9,9916058,None


In [106]:
table_df = test_query()

In [107]:
table_df.movie_id

0     9916730
1     9916680
2     9916538
3     9916190
4     9916186
5     9916160
6     9916134
7     9916132
8     9916124
9     9916058
10    9915946
11    9915872
12    9915130
13    9914942
14    9914928
15    9914828
16    9914644
17    9914642
18    9914582
19    9914286
20    9914254
21    9913936
22    9913872
23    9913660
24    9913594
25    9913320
26    9913084
27    9913056
28    9911774
29    9911750
Name: movie_id, dtype: object

In [88]:
full_df = pd.read_csv('movieid.csv', header=None)
id_col = full_df[1]
full_id_col = pd.DataFrame({'ids': remove_tt(id_col)})
matching_ids = pd.merge(table_df, full_id_col, how)

,ids
0,0288772
1,9042520
2,2911668
3,3489334
4,0814314
...,...
267416,6078532
267417,0371586
267418,0222195
267419,8255450


In [7]:
def add_url(table_df, url_df):
    new_table = pd.merge(table_df, url_df, how='left', left_on='movie_id', right_on='poster_url')
    return new_table

In [8]:
def loopscoop(id_list):
    poster_url = []
    movie_id = []
    
    for count, id in enumerate(id_list):
        tt_removed = id.replace("tt", "")
        movie_id.append(tt_removed)
        try:            
            result = tmdb.Find(f"{id}").info(external_source='imdb_id')['movie_results'][0]['poster_path']
        except Exception:
            poster_url.append("None")
            continue
        if result is None:
            poster_url.append('None')
            continue
        else:
            poster_url.append(result)
            continue
        
    url_df = pd.DataFrame(
            {
                'movie_id' : movie_id,
                'poster_url' : poster_url
                })
    return url_df

In [108]:
url_df = loopscoop(table_df.movie_id)

In [109]:
url_df

,movie_id,poster_url
0,9916730,None
1,9916680,None
2,9916538,None
3,9916190,None
4,9916186,None
5,9916160,None
6,9916134,None
7,9916132,None
8,9916124,None
9,9916058,None


In [112]:
result = tmdb.Find("9915130").info(external_source='imdb_id')['movie_results'][0]['poster_path']

IndexError: list index out of range

In [11]:
new_table = add_url(table_df, url_df)

In [12]:
new_table

,movie_id_x,poster_url_x,movie_id_y,poster_url_y
0,0000502,None,NaN,NaN
1,0000679,None,NaN,NaN
2,0001756,None,NaN,NaN
3,0002101,None,NaN,NaN
4,0002315,None,NaN,NaN
5,0002423,None,NaN,NaN
6,0002445,None,NaN,NaN
7,0002452,None,NaN,NaN
8,0002625,None,NaN,NaN
9,0002646,None,NaN,NaN
